In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from msslib.utils import *

import random

from pystruct.models import ChainCRF
import pystruct.learners as ssvm
from pystruct.utils import SaveLogger

from sklearn.decomposition import PCA

import datetime
import random

In [9]:
test_runs = ["B","C","D","E","F"]

In [10]:
unwanted_recto = ["VA223RN","VA327RN","VA001RN","VA002RN","VA006RN","VA007RN","VA008RN","VA009RN"]

def get_set_paths(directory):
    return sorted(list(filter(lambda x: "RN" in x and not any(s in x for s in unwanted_recto) , listpaths(directory))))

In [11]:
label_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/"
data_dir = "/home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/data/"    

label_paths = get_set_paths(label_dir)
data_paths = get_set_paths(data_dir)

path_pairs = list(zip(label_paths, data_paths))
print(len(path_pairs))

150


In [12]:
spir = Spiraliser(60,45)

In [13]:
for letter in test_runs:
    split_at = 20
    random.shuffle(path_pairs)
    train = path_pairs[:split_at]
    test = path_pairs[split_at:]
    
    oot_path = os.path.join("/home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/PCA_20_Spiral_Chain_CRF", letter)
    mkdir(oot_path)
 
    crf = ChainCRF()
    clf = ssvm.OneSlackSSVM(model=crf,n_jobs=-1, C=100, inference_cache=100, tol=.1,
                           logger=SaveLogger(oot_path+'.pickle', save_every=100))
    
    pca = PCA(n_components=20)
    print(datetime.datetime.now())
    print("Training PCA %s" %letter)
    for l, d in train:
        X = spir.flatten(np.load(d))
        #y = np.squeeze(flatten_vector_matrix(np.load(l)))
        pca.fit(X)
  
    print("Training %s" %letter)
    train_X = []
    train_y = []
    for l, d in train:
        print("Training on %s" %l)
        train_X.append(pca.transform(spir.flatten(np.load(d))))
        train_y.append(np.squeeze(spir.flatten(np.load(l)).astype('int')))
    print(np.array(train_X).shape)
    clf.fit(train_X, train_y)
        
    print("Testing %s" %letter)
    test_results = []
    for l, d in test:
        X = pca.transform(spir.flatten(np.load(d)))
        pred = np.asarray(clf.predict([X]))
        test_results.append({
            'name': only_basename(l),
            'pred': spir.reshape(pred[0]),
            })
    
    print("Saving %s" %letter)
    for result in test_results:
        path = format_path(oot_path, 'npy', result['name'])
        np.save(path, scale_matrix(result['pred'], 20))
    print(datetime.datetime.now())


2016-06-07 14:24:00.770794
Training PCA B
Training B
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA056RN-0057.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA214RN-0215.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA018RN-0019.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA091RN-0092.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA248RN-0249.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA175RN-0176.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA080RN-0081.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA116RN-0288.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA016RN-0017.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA305RN-0475.npy
Training on /home/finlay/HMT/data/VenetusA/1200Hx900W/20blocks/labels/VA225RN

In [15]:
900/15


60.0

In [ ]:
train_X.shape

In [ ]:
wit = np.load(d)
pwit = pca.transform(spir.flatten(wit))

In [ ]:
pred = clf.predict([pwit])

In [ ]:
plt.imshow(spir.reshape(pred[0]))